In [1]:
!pip install ultralytics roboflow albumentations opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 41.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [2]:
import os
from google.colab import files

print("Будь ласка, завантажте файл kaggle.json:")
uploaded = files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("\nЗавантаження датасету kovalkostiantyn/mkp-data...")
!kaggle datasets download -d kovalkostiantyn/mkp-data
dataset_dir = "/content/dataset"
if os.path.exists(dataset_dir):
    import shutil
    shutil.rmtree(dataset_dir)
os.makedirs(dataset_dir, exist_ok=True)

!unzip -q mkp-data.zip -d {dataset_dir}
print("Готово!")






import yaml
import glob

yaml_files = glob.glob(f"{dataset_dir}/**/data.yaml", recursive=True)

if yaml_files:
    yaml_path = yaml_files[0]
    base_path = os.path.dirname(yaml_path)

    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    data['path'] = base_path
    data['train'] = "train/images"
    data['val'] = "valid/images"
    if 'test' in data:
        data['test'] = "test/images"

    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)

    print("Файл data.yaml успішно оновлено для роботи в Colab!")
    print(f"Класи: {data['names']}")
else:
    print("ПОМИЛКА: Не знайдено файл data.yaml у завантаженому архіві!")






import albumentations as A
import cv2
import matplotlib.pyplot as plt
import random

train_images_dir = os.path.join(base_path, "train", "images")
image_files = os.listdir(train_images_dir)

if image_files:
    rand_img = random.choice(image_files)
    img_path = os.path.join(train_images_dir, rand_img)

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 7 методів аугментації
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Rotate(limit=20, p=0.5),
        A.GaussianBlur(p=0.2),
        A.RGBShift(p=0.2),
        A.RandomFog(p=0.1),
        A.CLAHE(p=0.5),
        A.ChannelShuffle(p=0.1)
    ])

else:
    print("Папка з зображеннями порожня!")

Будь ласка, завантажте файл kaggle.json:


Saving kaggle.json to kaggle.json

Завантаження датасету kovalkostiantyn/mkp-data...
Dataset URL: https://www.kaggle.com/datasets/kovalkostiantyn/mkp-data
License(s): apache-2.0
  0% 0.00/7.00M [00:00<?, ?B/s]
100% 7.00M/7.00M [00:00<00:00, 1.33GB/s]
Готово!
Файл data.yaml успішно оновлено для роботи в Colab!
Класи: ['lexus-logo']


In [3]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')

results = model.train(
    data='/content/dataset/data.yaml',
    epochs=100,           #
    patience=0,
    batch=8,
    imgsz=640,
    name='mkr_neiro',
    optimizer='AdamW',
    lr0=0.001,
    exist_ok=True
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=Fals

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:832: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


      2/100      1.26G      1.295      4.073     0.9808         12        640: 100% ━━━━━━━━━━━━ 7/7 6.1it/s 1.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 24.5it/s 0.1s
                   all         18          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:832: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      3/100      1.27G      1.134      2.729     0.9724         11        640: 100% ━━━━━━━━━━━━ 7/7 7.4it/s 0.9s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 16.3it/s 0.1s
                   all         18          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    100/100       2.7G     0.5752     0.4777     0.8153          5        640: 100% ━━━━━━━━━━━━ 7/7 7.5it/s 0.9s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 22.7it/s 0.1s
                   all         18          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:832: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



100 epochs completed in 0.049 hours.
Optimizer stripped from /content/runs/detect/mkr_neiro/weights/last.pt, 5.4MB
Optimizer stripped from /content/runs/detect/mkr_neiro/weights/best.pt, 5.4MB

Validating /content/runs/detect/mkr_neiro/weights/best.pt...
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 11.3it/s 0.2s


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:653: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all         18          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels
Speed: 0.1ms preprocess, 6.1ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /content/runs/detect/mkr_neiro


In [6]:
from google.colab import files
import os

video_path = "test_video.mp4"

if not os.path.exists(video_path):
    print("Завантажте відеофайл!")
    uploaded = files.upload()
    video_path = list(uploaded.keys())[0]

best_weights = "/content/runs/detect/mkr_neiro/weights/best.pt"

if os.path.exists(best_weights):
    print(f"Завантаження моделі: {best_weights}")
    model = YOLO(best_weights)

    results = model.predict(source=video_path, save=True, conf=0.69)

    print("\nВідео оброблено успішно!")

    import glob
    latest_predict = sorted(glob.glob('/content/runs/detect/predict*'))[-1]
    print(f"Результат збережено в: {latest_predict}")

else:
    print("Помилка: Файл ваг best.pt не знайдено. Навчання не завершилося успішно.")

Завантаження моделі: /content/runs/detect/mkr_neiro/weights/best.pt

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/3238) /content/test_video.mp4: 384x640 (no detections), 10.9ms
video 1/1 (frame 2/3238) /content/test_video.mp4: 384x640 (no detections), 8.9ms
video 1/1 (frame 3/3238) /content/test_video.mp4: 384x640 (no detections), 8.7ms
video 1/1 (frame 4/3238) /content/test_video.mp4: 384x640 (no detections), 8.6ms
video 1/1 (frame 5/3238) /content/test_video.mp4: 384x640 (no d